In [ ]:
#Connecting with google drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !ls gdrive/MyDrive/generate_content/code
%cd gdrive/MyDrive/generate_content/code

# !mkdir data
# !mkdir data/poetry
# !mkdir data/poetry/trainer

/content/gdrive/MyDrive/generate_content/code


In [ ]:
%%bash
pip freeze | grep tensor

pytensor==2.10.1
tensorboard==2.11.2
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.1
tensorflow==2.11.0
tensorflow-datasets==4.8.3
tensorflow-estimator==2.11.0
tensorflow-gcs-config==2.11.0
tensorflow-hub==0.13.0
tensorflow-io-gcs-filesystem==0.31.0
tensorflow-metadata==1.12.0
tensorflow-probability==0.19.0
tensorstore==0.1.33


In [ ]:
%%bash
pip install tensor2tensor==1.13.1 tensorflow==1.13.1 tensorflow-serving-api==1.13# gutenberg
pip install tensorflow_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


ERROR: Could not find a version that satisfies the requirement tensorflow==1.13.1 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==1.13.1


In [ ]:
%%bash
pip freeze | grep tensor

pytensor==2.10.1
tensorboard==2.11.2
tensorboard-data-server==0.6.1
tensorboard-plugin-wit==1.8.1
tensorflow==2.11.0
tensorflow-datasets==4.8.3
tensorflow-estimator==2.11.0
tensorflow-gcs-config==2.11.0
tensorflow-hub==0.13.0
tensorflow-io-gcs-filesystem==0.31.0
tensorflow-metadata==1.12.0
tensorflow-probability==0.19.0
tensorstore==0.1.33


In [ ]:
!sudo apt install libdb5.3-dev
!sudo pip install gutenberg

Reading package lists... Done
Building dependency tree       
Reading state information... Done
Suggested packages:
  db5.3-doc
The following NEW packages will be installed:
  libdb5.3-dev
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 766 kB of archives.
After this operation, 3,210 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 libdb5.3-dev amd64 5.3.28+dfsg1-0.6ubuntu2 [766 kB]
Fetched 766 kB in 0s (2,735 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package libdb5.3-dev.


In [ ]:
#Downloading the data

# Keep it in seperate cell only
# Run it twice
from gutenberg.acquire import load_etext

In [ ]:
from gutenberg.cleanup import strip_headers
import re

books = [
  # bookid, skip N lines
  (26715, 1000, 'Victorian songs'),
  (30235, 580, 'Baldwin collection'),
  (35402, 710, 'Swinburne collection'),
  (574, 15, 'Blake'),
  (1304, 172, 'Bulchevys collection'),
  (19221, 223, 'Palgrave-Pearse collection'),
  (15553, 522, 'Knowles collection')
]

In [ ]:
with open('data/poetry/raw.txt', 'w') as ofp:
  lineno = 0
  for (id_nr, toskip, title) in books:
    startline = lineno
    text = strip_headers(load_etext(id_nr,mirror = "http://mirrors.xmission.com/gutenberg/")).strip()
    lines = text.split('\n')[toskip:]
    # any line that is all upper case is a title or author name
    # also don't want any lines with years (numbers)
    for line in lines:
      if (len(line) > 0
          and line.upper() != line
          and not re.match('.*[0-9]+.*', line)
          and len(line) < 50
         ):
        cleaned = re.sub('[^a-z\'\-]+', ' ', line.strip().lower())
        ofp.write(cleaned)
        ofp.write('\n')
        lineno = lineno + 1
      else:
        ofp.write('\n')
    print('Wrote lines {} to {} from {}'.format(startline, lineno, title))

Wrote lines 0 to 2802 from Victorian songs
Wrote lines 2802 to 7503 from Baldwin collection
Wrote lines 7503 to 14069 from Swinburne collection
Wrote lines 14069 to 14926 from Blake
Wrote lines 14926 to 42441 from Bulchevys collection
Wrote lines 42441 to 51789 from Palgrave-Pearse collection
Wrote lines 51789 to 56298 from Knowles collection


In [ ]:
#creating the data
with open('data/poetry/raw.txt', 'r') as rawfp,\
  open('data/poetry/input.txt', 'w') as infp,\
  open('data/poetry/output.txt', 'w') as outfp:

    prev_line = ''
    for curr_line in rawfp:
        curr_line = curr_line.strip()
        # poems break at empty lines, so this ensures we train only
        # on lines of the same poem
        if len(prev_line) > 0 and len(curr_line) > 0:
            infp.write(prev_line + '\n')
            outfp.write(curr_line + '\n')
        prev_line = curr_line

In [ ]:
!head -5 data/poetry/*.txt

==> data/poetry/input.txt <==
i sat beside the streamlet
i watched the water flow
as we together watched it
one little year ago
the soft rain pattered on the leaves

==> data/poetry/output.txt <==
i watched the water flow
as we together watched it
one little year ago
the soft rain pattered on the leaves
the april grass was wet

==> data/poetry/raw.txt <==







In [ ]:
#creating the dataframe
ind=0
poem_stanzas=[]

poem_lines=[]

with open('data/poetry/raw.txt', 'r') as rawfp:
  prev_line = ''
  curr_stanza=""
  for curr_line in rawfp:
    curr_line = curr_line.strip()
    # poems break at empty lines, so this ensures we train only
    # on lines of the same poem
    if len(prev_line) > 0 and len(curr_line) > 0:
      curr_stanza=curr_stanza+curr_line+"\n"
    else:
      if len(curr_stanza) > 0:
        poem_stanzas.append(curr_stanza)
      curr_stanza=""

    prev_line = curr_line
    # ind+=1

    # if ind==100:
    #   break

In [ ]:
import pandas as pd

In [ ]:
df_stanza=pd.DataFrame({"stanza_num":[x for x in range(len(poem_stanzas))], "stanza":poem_stanzas})

In [ ]:
df_stanza.to_csv("data/poetry/poem_stanza.csv",index=False)

In [ ]:
!mkdir poetry/trainer

mkdir: cannot create directory ‘poetry/trainer’: No such file or directory
